<a href="https://colab.research.google.com/github/Darkcoder1995/Graph-Neural-Network/blob/main/GraphSAGERegressor_ETA_Pred_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch-geometric

In [ ]:
!pip install torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
  Using cached https://data.pyg.org/whl/torch-2.6.0%2Bcu124/torch_scatter-2.1.2%2Bpt26cu124-cp311-cp311-linux_x86_64.whl (10.8 MB)
  Using cached https://data.pyg.org/whl/torch-2.6.0%2Bcu124/torch_sparse-0.6.18%2Bpt26cu124-cp311-cp311-linux_x86_64.whl (5.0 MB)


In [ ]:
!pip install sodapy

In [ ]:
!pip install openrouteservice

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

import torch.nn as nn
import torch.nn.functional as F
# from torch_geometric.nn import SAGEConv, GATConv
from torch_geometric.loader import NeighborLoader
from sodapy import Socrata
from geopy.distance import geodesic

from sklearn.neighbors import BallTree
import numpy as np
import requests
import time

import openrouteservice
from tqdm import tqdm
from openrouteservice import Client
from openrouteservice.exceptions import ApiError
import time
from sklearn.preprocessing import StandardScaler
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import softmax
import torch.optim as optim
from torch_scatter import scatter
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
from xgboost import XGBRegressor

In [ ]:
# # # # prompt: read results_df_final csv file
# #
results_df_final = pd.read_csv('processed_taxi_data (1).csv')

In [ ]:
# Define weather columns (pickup + dropoff)
weather_columns = [
    'pickup_PRCP', 'pickup_SNOW', 'pickup_SNWD', 'pickup_WESF', 'pickup_WESD', 'pickup_DAPR',
    'pickup_MDPR', 'pickup_ADPT', 'pickup_ASLP', 'pickup_ASTP', 'pickup_AWBT', 'pickup_AWND',
    'pickup_RHAV', 'pickup_RHMN', 'pickup_RHMX', 'pickup_TMAX', 'pickup_TMIN', 'pickup_WDF2',
    'pickup_WDF5', 'pickup_WSF2', 'pickup_WSF5', 'pickup_WT01', 'pickup_WT08', 'pickup_WT04',
    'pickup_WT02', 'pickup_WT06', 'pickup_TAVG', 'pickup_WT09', 'pickup_WT03',
    'dropoff_PRCP', 'dropoff_SNOW', 'dropoff_SNWD', 'dropoff_WESF', 'dropoff_WESD', 'dropoff_DAPR',
    'dropoff_MDPR', 'dropoff_ADPT', 'dropoff_ASLP', 'dropoff_ASTP', 'dropoff_AWBT', 'dropoff_AWND',
    'dropoff_RHAV', 'dropoff_RHMN', 'dropoff_RHMX', 'dropoff_TMAX', 'dropoff_TMIN', 'dropoff_WDF2',
    'dropoff_WDF5', 'dropoff_WSF2', 'dropoff_WSF5', 'dropoff_WT01', 'dropoff_WT08', 'dropoff_WT04',
    'dropoff_WT02', 'dropoff_WT06', 'dropoff_TAVG', 'dropoff_WT09', 'dropoff_WT03'
]

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Combine trip-related and weather columns
scale_columns = [
    'trip_distance', 'passenger_count', 'fare_amount', 'tip_amount',
    'total_amount', 'ors_distance_m'
] + weather_columns  # Include weather data

# Ensure numeric types (coerce non-numeric to NaN)
results_df_final[scale_columns] = results_df_final[scale_columns].apply(pd.to_numeric, errors='coerce')

# Remove or replace invalid (negative or NaN) values
# Option 1: Fill NaNs with small value before log (safer for log1p)
results_df_final[scale_columns] = results_df_final[scale_columns].fillna(0)

# Optionally clip negatives if log1p is not safe for < -1
results_df_final[scale_columns] = results_df_final[scale_columns].clip(lower=0)

# Step 1: Apply log1p
results_df_final_log = results_df_final.copy()
results_df_final_log[scale_columns] = np.log1p(results_df_final_log[scale_columns])

# Step 2: Scale
scaler = StandardScaler()
scaled_features = scaler.fit_transform(results_df_final_log[scale_columns])

# Step 3: Replace the original values with the scaled ones
results_df_final[scale_columns] = scaled_features

In [ ]:
# # Ensure datetime columns are parsed correctly
# results_df_final['lpep_pickup_datetime'] = pd.to_datetime(results_df_final['lpep_pickup_datetime'])
# results_df_final['lpep_dropoff_datetime'] = pd.to_datetime(results_df_final['lpep_dropoff_datetime'])

# # Calculate trip duration in minutes
# results_df_final['trip_duration'] = (
#     results_df_final['lpep_dropoff_datetime'] - results_df_final['lpep_pickup_datetime']
# ).dt.total_seconds() / 60

# # Extract weather-related columns
# weather_columns = [col for col in results_df_final.columns if col.startswith('pickup_') or col.startswith('dropoff_')]

# from xgboost import XGBRegressor
# import numpy as np

# # Define features and target
# # Exclude the target variable 'trip_duration' and any identifier columns from features
# features_columns = [col for col in results_df_final.columns if col not in ['trip_duration', 'lpep_pickup_datetime', 'lpep_dropoff_datetime']]

# X = results_df_final[features_columns]
# y = results_df_final['trip_duration']

# # Ensure feature and target are in numpy format (XGBoost can also work with pandas DataFrames)
# X_np = X.values
# y_np = y.values

# # Train XGBoost
# xgb_model = XGBRegressor()
# xgb_model.fit(X_np, y_np)

# # Get feature importances
# importances = xgb_model.feature_importances_

# # Select top features (optional, based on your need)
# # You can adjust top_k as needed
# top_k = 25
# top_indices = np.argsort(importances)[::-1][:top_k]

# # Get the names of the top features
# top_features_names = [features_columns[i] for i in top_indices]

# print("Top Features (XGBoost):", top_features_names)

# # If you want to use only the top features for further processing:
# X_selected_np = X_np[:, top_indices]

# print(f"Shape of selected features: {X_selected_np.shape}")

# features = []
# targets = []

# # Feature construction
# for i, row in results_df_final.iterrows():
#     try:
#         base_features = [
#             pd.to_numeric(row['trip_distance'], errors='coerce'),
#             pd.to_numeric(row['passenger_count'], errors='coerce'),
#             pd.to_numeric(row['fare_amount'], errors='coerce'),
#             pd.to_numeric(row['tip_amount'], errors='coerce'),
#             pd.to_numeric(row['total_amount'], errors='coerce'),
#             pd.to_numeric(row['ors_distance_m'], errors='coerce'),
#             pd.to_numeric(row['ors_duration_s'], errors='coerce')
#         ]

#         weather_features = [pd.to_numeric(row[col], errors='coerce') for col in weather_columns]
#         feature_vector = [0 if pd.isna(f) else f for f in base_features + weather_features]

#         features.append(feature_vector)
#         targets.append(row['trip_duration'])

#     except Exception as e:
#         print(f"❌ Skipping row {i} due to error: {e}")

In [ ]:
# Ensure datetime columns are parsed correctly
results_df_final['lpep_pickup_datetime'] = pd.to_datetime(results_df_final['lpep_pickup_datetime'])
results_df_final['lpep_dropoff_datetime'] = pd.to_datetime(results_df_final['lpep_dropoff_datetime'])

# Calculate trip duration in minutes
results_df_final['trip_duration'] = (
    results_df_final['lpep_dropoff_datetime'] - results_df_final['lpep_pickup_datetime']
).dt.total_seconds() / 60

# Extract weather-related columns (already defined earlier, but redefined here)
# This is safe as it's just defining a list of column names.
# weather_columns = [col for col in results_df_final.columns if col.startswith('pickup_') or col.startswith('dropoff_')]

from xgboost import XGBRegressor
import numpy as np

# Define features and target
# Exclude the target variable 'trip_duration' and any identifier columns from features
features_columns = [col for col in results_df_final.columns if col not in ['trip_duration', 'lpep_pickup_datetime', 'lpep_dropoff_datetime']]

# --- Start of suggested changes ---

# Ensure all feature columns are numeric and handle missing/invalid values
for col in features_columns:
    # Convert column to numeric, coercing errors to NaN
    results_df_final[col] = pd.to_numeric(results_df_final[col], errors='coerce')

# Fill remaining NaNs with a suitable value (e.g., 0 or mean/median)
# Using 0 as in your previous scaling step for consistency, but consider alternatives if appropriate
results_df_final[features_columns] = results_df_final[features_columns].fillna(0)

# Optionally clip negative values if they don't make sense for certain features
results_df_final[features_columns] = results_df_final[features_columns].clip(lower=0)

# --- End of suggested changes ---


X = results_df_final[features_columns]
y = results_df_final['trip_duration']

# Ensure feature and target are in numpy format (XGBoost can also work with pandas DataFrames)
X_np = X.values
y_np = y.values

# Train XGBoost
xgb_model = XGBRegressor()
xgb_model.fit(X_np, y_np)

# Get feature importances
importances = xgb_model.feature_importances_

# Select top features (optional, based on your need)
# You can adjust top_k as needed
top_k = 65
top_indices = np.argsort(importances)[::-1][:top_k]

# Get the names of the top features
top_features_names = [features_columns[i] for i in top_indices]

print("Top Features (XGBoost):", top_features_names)

# If you want to use only the top features for further processing:
X_selected_np = X_np[:, top_indices]

print(f"Shape of selected features: {X_selected_np.shape}")

# The following feature construction loop is no longer necessary for preparing data
# for the Autoencoder if you use X_selected_np directly. However, if you intend
# to use the 'features' and 'targets' lists for something else later, you might
# need to adjust this loop based on the cleaned DataFrame. Assuming you want to
# use the cleaned `X_selected_np` and `y_np` for the subsequent Autoencoder step:

# features = [] # No longer needed if using X_selected_np
targets = [] # No longer needed if using y_np

# # Feature construction (commented out as X_selected_np should be used instead)
for i, row in results_df_final.iterrows():
    try:
#         base_features = [
#             pd.to_numeric(row['trip_distance'], errors='coerce'),
#             pd.to_numeric(row['passenger_count'], errors='coerce'),
#             pd.to_numeric(row['fare_amount'], errors='coerce'),
#             pd.to_numeric(row['tip_amount'], errors='coerce'),
#             pd.to_numeric(row['total_amount'], errors='coerce'),
#             pd.to_numeric(row['ors_distance_m'], errors='coerce'),
#             pd.to_numeric(row['ors_duration_s'], errors='coerce')
#         ]

#         weather_features = [pd.to_numeric(row[col], errors='coerce') for col in weather_columns]
#         feature_vector = [0 if pd.isna(f) else f for f in base_features + weather_features]

#         features.append(feature_vector)
        targets.append(row['trip_duration'])

    except Exception as e:
        print(f"❌ Skipping row {i} due to error: {e}")

Top Features (XGBoost): ['pickup_SNOW', 'pickup_AWND', 'dropoff_PRCP', 'pickup_PRCP', 'payment_type', 'dropoff_latitude', 'extra', 'pickup_station_lat', 'total_amount', 'ors_duration_s', 'fare_amount', 'pickup_latitude', 'ors_distance_m', 'trip_distance', 'dropoff_SNOW', 'passenger_count', 'tip_amount', 'dropoff_RHAV', 'dropoff_station_lat', 'vendorid', 'ratecodeid', 'pickup_TMIN', 'dropoff_RHMX', 'pickup_WDF5', 'dropoff_WDF5', 'trip_type', 'dropoff_TMIN', 'dropoff_WSF5', 'mta_tax', 'pickup_WSF2', 'dropoff_TMAX', 'improvement_surcharge', 'dropoff_WDF2', 'tolls_amount', 'dropoff_AWND', 'dropoff_WSF2', 'pickup_WSF5', 'pickup_TMAX', 'dropoff_SNWD', 'pickup_WDF2', 'dropoff_ASLP', 'pickup_SNWD', 'dropoff_ASTP', 'dropoff_TAVG', 'pickup_ASLP', 'pickup_RHAV', 'dropoff_RHMN', 'pickup_AWBT', 'pickup_WESF', 'pickup_TAVG', 'dropoff_WT01', 'dropoff_WT03', 'dropoff_WT09', 'dropoff_WT06', 'dropoff_AWBT', 'dropoff_WT08', 'dropoff_WT04', 'dropoff_WT02', 'pickup_MDPR', 'pickup_RHMX', 'dropoff_DAPR', 'dr

In [ ]:
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Define the Autoencoder class
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim=32):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
try:
    features_np = np.array(X_selected_np, dtype=np.float32)

    input_dim = features_np.shape[1]
    encoding_dim = top_k  # Adjust as needed

    # Instantiate model and prepare for training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ae = Autoencoder(input_dim, encoding_dim).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(ae.parameters(), lr=1e-3)

    # Prepare data loader
    X_tensor = torch.tensor(features_np, dtype=torch.float32).to(device)
    dataset = TensorDataset(X_tensor)
    loader = DataLoader(dataset, batch_size=64, shuffle=True)

    # Training loop
    for epoch in range(50):
        ae.train()
        total_loss = 0
        for batch in loader:
            inputs = batch[0]
            optimizer.zero_grad()
            encoded, decoded = ae(inputs)
            loss = criterion(decoded, inputs)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

    # Extract encoded features
    ae.eval()
    with torch.no_grad():
        features_encoded, _ = ae(X_tensor)

    X = features_encoded.cpu()
    y = torch.tensor(targets, dtype=torch.float32)

    print("✅ Encoded feature tensor shape (Autoencoder):", X.shape)
    print("✅ Target tensor shape:", y.shape)

except ValueError as e:
    print(f"❌ Error converting features to numpy array: {e}")

Epoch 1, Loss: 1514.4522
Epoch 2, Loss: 47.2993
Epoch 3, Loss: 28.8893
Epoch 4, Loss: 19.2787
Epoch 5, Loss: 15.7135
Epoch 6, Loss: 13.3067
Epoch 7, Loss: 10.8400
Epoch 8, Loss: 8.7416
Epoch 9, Loss: 7.3090
Epoch 10, Loss: 7.1839
Epoch 11, Loss: 6.1528
Epoch 12, Loss: 6.8377
Epoch 13, Loss: 5.0938
Epoch 14, Loss: 4.8743
Epoch 15, Loss: 4.8764
Epoch 16, Loss: 4.6775
Epoch 17, Loss: 3.7048
Epoch 18, Loss: 3.4784
Epoch 19, Loss: 3.7160
Epoch 20, Loss: 3.3902
Epoch 21, Loss: 3.5679
Epoch 22, Loss: 3.0015
Epoch 23, Loss: 3.0245
Epoch 24, Loss: 3.3158
Epoch 25, Loss: 3.2400
Epoch 26, Loss: 2.6027
Epoch 27, Loss: 2.7115
Epoch 28, Loss: 2.7305
Epoch 29, Loss: 3.3720
Epoch 30, Loss: 2.8898
Epoch 31, Loss: 3.0946
Epoch 32, Loss: 2.3285
Epoch 33, Loss: 2.1895
Epoch 34, Loss: 1.9899
Epoch 35, Loss: 1.8493
Epoch 36, Loss: 1.7864
Epoch 37, Loss: 1.9266
Epoch 38, Loss: 2.2651
Epoch 39, Loss: 3.0349
Epoch 40, Loss: 1.5874
Epoch 41, Loss: 2.4347
Epoch 42, Loss: 1.4646
Epoch 43, Loss: 1.8878
Epoch 44, L

In [ ]:
# # X is already the encoded output from autoencoder
# encoded_np = X.numpy()

# # Make sure targets is in numpy format
# targets_np = np.array(targets)

# # Train XGBoost on encoded features
# xgb_model = XGBRegressor()
# xgb_model.fit(encoded_np, targets_np)

# # Select top features
# importances = xgb_model.feature_importances_
# top_k = 25
# top_indices = np.argsort(importances)[::-1][:top_k]
# selected_encoded = encoded_np[:, top_indices]

# # Convert to tensor for further use
# X_tensor_selected = torch.tensor(selected_encoded, dtype=torch.float32)

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Ensure X_tensor is on CPU for sklearn
X_tensor = X.cpu()
X_np = X_tensor.numpy()  # Convert to NumPy for NearestNeighbors

# k-NN graph (skip self-loop)
k = 10
nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(X_np)
distances, neighbors = nbrs.kneighbors(X_np)

edge_index_list = []
edge_attr_list = []

for i in range(X_np.shape[0]):
    for idx, j in enumerate(neighbors[i, 1:]):  # skip self-edge (neighbors[i, 0])
        edge_index_list.append([i, j])
        edge_attr_list.append(distances[i, idx + 1])  # +1 to skip self-distance

# Convert to tensors
edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()
edge_attr = torch.tensor(edge_attr_list, dtype=torch.float)

print(f"✅ Edge Index Shape: {edge_index.shape}")  # [2, num_edges]
print(f"✅ Edge Attr Shape: {edge_attr.shape}")    # [num_edges]

✅ Edge Index Shape: torch.Size([2, 100000])
✅ Edge Attr Shape: torch.Size([100000])


In [ ]:
# Ensure `X`, `edge_index`, `edge_attr`, and labels `y` exist
y_tensor = torch.tensor(targets, dtype=torch.float)
y_tensor = torch.log1p(y_tensor)

# Ensure edge_index is correctly formatted
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)

# Ensure edge attributes exist
edge_attr_tensor = torch.tensor(edge_attr, dtype=torch.float)

# ✅ Create PyTorch Geometric Data object (fixes the issue)
data = Data(
    x=X_tensor,  # Node features
    edge_index=edge_index_tensor,  # Connectivity between nodes
    edge_attr=edge_attr_tensor,  # Edge attributes (distances)
    y=y_tensor  # Target trip duration values
)

print(f"✅ Data object created! Node shape: {data.x.shape}, Edge shape: {data.edge_index.shape}")

✅ Data object created! Node shape: torch.Size([10000, 65]), Edge shape: torch.Size([2, 100000])


<ipython-input-53-adfac7b809ef>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)
<ipython-input-53-adfac7b809ef>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr_tensor = torch.tensor(edge_attr, dtype=torch.float)


In [ ]:
from sklearn.model_selection import train_test_split

# Get total node count
num_nodes = data.x.shape[0]

# Generate indices and split into train/test (80/20)
indices = np.arange(num_nodes)
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

# Define boolean masks for PyTorch Geometric
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_indices] = True
test_mask[test_indices] = True

# Assign masks to `Data` object
data.train_mask = train_mask
data.test_mask = test_mask

print(f"✅ Train/Test masks applied! Train nodes: {train_mask.sum().item()}, Test nodes: {test_mask.sum().item()}")

✅ Train/Test masks applied! Train nodes: 8000, Test nodes: 2000


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SAGEConv(nn.Module):
    def __init__(self, in_channels, out_channels, aggr='mean'):
        super(SAGEConv, self).__init__()
        self.aggr = aggr
        self.lin = nn.Linear(in_channels * 2, out_channels)

    def forward(self, x, edge_index):
        row, col = edge_index

        # Aggregate neighbor messages
        if self.aggr == 'mean':
            deg = torch.bincount(row, minlength=x.size(0)).clamp(min=1)
            aggr_messages = torch.zeros_like(x)
            aggr_messages.index_add_(0, row, x[col])
            aggr_messages = aggr_messages / deg.unsqueeze(-1)
        else:
            raise NotImplementedError(f"Aggregation '{self.aggr}' not implemented.")

        # Concatenate with self-node representation and apply linear transformation
        out = self.lin(torch.cat([x, aggr_messages], dim=1))
        return out


In [ ]:
class GATConv(nn.Module):
    def __init__(self, in_channels, out_channels, heads=1, concat=True, dropout=0.0, negative_slope=0.2):
        super(GATConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.dropout = dropout
        self.negative_slope = negative_slope

        self.lin = nn.Linear(in_channels, heads * out_channels, bias=False)
        self.attn = nn.Parameter(torch.Tensor(1, heads, 2 * out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.lin.weight)
        nn.init.xavier_uniform_(self.attn)

    def forward(self, x, edge_index):
        H, C = self.heads, self.out_channels

        x = self.lin(x).view(-1, H, C)  # (N, H, C)
        row, col = edge_index

        x_i = x[row]  # target node features
        x_j = x[col]  # source node features

        # Concatenate node pairs and compute attention scores
        alpha = torch.cat([x_i, x_j], dim=-1)  # (E, H, 2C)
        alpha = (alpha * self.attn).sum(dim=-1)  # (E, H)
        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = F.softmax(alpha, dim=0)  # Softmax over all edges globally (simplified)

        alpha = F.dropout(alpha, p=self.dropout, training=self.training)

        # Weight features by attention scores and aggregate
        out = torch.zeros(x.size(0), H, C, device=x.device)
        out.index_add_(0, row, x_j * alpha.unsqueeze(-1))  # attention-weighted sum

        if self.concat:
            out = out.view(-1, H * C)
        else:
            out = out.mean(dim=1)  # average over heads

        return out


In [ ]:
# Define your model
class GraphSAGERegressor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5, use_attention=False):
        super(GraphSAGERegressor, self).__init__()
        self.use_attention = use_attention
        self.dropout = dropout

        self.convs = nn.ModuleList()
        if use_attention:
            self.convs.append(GATConv(in_channels, hidden_channels))
            self.convs.append(GATConv(hidden_channels, hidden_channels))
        else:
            self.convs.append(SAGEConv(in_channels, hidden_channels))
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))

        self.lin = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        return self.lin(x).view(-1)

In [ ]:
# Prepare device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Send the full graph data to the device
data = data.to(device)

# Initialize model, optimizer, and loss
model = GraphSAGERegressor(
    in_channels=data.x.shape[1],
    hidden_channels=1024,
    out_channels=1,
    dropout=0.3,
    use_attention=True
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.MSELoss()

# Training loop
for epoch in range(1, 201):
    model.train()
    optimizer.zero_grad()

    out = model(data.x, data.edge_index)

    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Eval
    model.eval()
    with torch.no_grad():
        val_pred = model(data.x, data.edge_index)
        val_loss = loss_fn(val_pred[data.test_mask], data.y[data.test_mask])

    print(f"Epoch {epoch:03d} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

Epoch 001 | Train Loss: 6.1594 | Val Loss: 6.2295
Epoch 002 | Train Loss: 6.1556 | Val Loss: 6.2247
Epoch 003 | Train Loss: 6.1509 | Val Loss: 6.2200
Epoch 004 | Train Loss: 6.1461 | Val Loss: 6.2152
Epoch 005 | Train Loss: 6.1414 | Val Loss: 6.2104
Epoch 006 | Train Loss: 6.1366 | Val Loss: 6.2057
Epoch 007 | Train Loss: 6.1319 | Val Loss: 6.2009
Epoch 008 | Train Loss: 6.1272 | Val Loss: 6.1961
Epoch 009 | Train Loss: 6.1224 | Val Loss: 6.1914
Epoch 010 | Train Loss: 6.1177 | Val Loss: 6.1866
Epoch 011 | Train Loss: 6.1129 | Val Loss: 6.1819
Epoch 012 | Train Loss: 6.1082 | Val Loss: 6.1771
Epoch 013 | Train Loss: 6.1035 | Val Loss: 6.1724
Epoch 014 | Train Loss: 6.0988 | Val Loss: 6.1677
Epoch 015 | Train Loss: 6.0940 | Val Loss: 6.1629
Epoch 016 | Train Loss: 6.0893 | Val Loss: 6.1582
Epoch 017 | Train Loss: 6.0846 | Val Loss: 6.1534
Epoch 018 | Train Loss: 6.0799 | Val Loss: 6.1487
Epoch 019 | Train Loss: 6.0752 | Val Loss: 6.1440
Epoch 020 | Train Loss: 6.0704 | Val Loss: 6.1392


In [ ]:
# Put model in eval mode
model.eval()

# Predict on the whole data
with torch.no_grad():
    # Run the model on the full graph
    out = model(data.x, data.edge_index)

    # Filter predictions and targets using the test mask
    y_true_log = data.y[data.test_mask].cpu()
    y_pred_log = out[data.test_mask].cpu()

    # Invert log1p transform
    y_true = torch.expm1(y_true_log).numpy()
    y_pred = torch.expm1(y_pred_log).numpy()

# Compute regression metrics
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)

print(f"✅ MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

✅ MSE: 2755.1963, RMSE: 52.4900, MAE: 15.1501


In [ ]:

# ✅ MSE: 2472.2166, RMSE: 49.7214, MAE: 5.5530 LR=0.0005
# ✅ MSE: 2512.5269, RMSE: 50.1251, MAE: 7.5888 LR=0.0001
# ✅ MSE: 2736.7002, RMSE: 52.3135, MAE: 14.5382 LR=0.005
# ✅ MSE: 2713.2549, RMSE: 52.0889, MAE: 13.7352 LR=0.001
# ✅ MSE: 2753.5601, RMSE: 52.4744, MAE: 15.0962 LR=0.0001
# ✅ MSE: 2754.0442, RMSE: 52.4790, MAE: 15.1122 LR=0.0001
# ✅ MSE: 2551.4868, RMSE: 50.5122, MAE: 9.2425 LR=0.5
# ✅ MSE: 2551.4746, RMSE: 50.5121, MAE: 9.2425 LR=0.75
# ✅ MSE: 2551.4775, RMSE: 50.5122, MAE: 9.2425 LR=0.1
# ✅ MSE: 2551.4775, RMSE: 50.5122, MAE: 9.2425 LR=0.1
# INCREASED THE FACTORS FROM 45 TO 65
# ✅ MSE: 2754.9719, RMSE: 52.4878, MAE: 15.1427 LR=0.0001
# ✅ MSE: 2748.1340, RMSE: 52.4226, MAE: 14.9180 LR=0.001
# ✅ MSE: 2551.5813, RMSE: 50.5132, MAE: 9.2480 LR=0.05
# ✅ MSE: 2551.4761, RMSE: 50.5121, MAE: 9.2425 LR=0.1
#CHANGED LAYERS TO 256 FROM 512
# ✅ MSE: 2551.4780, RMSE: 50.5122, MAE: 9.2425 LR=0.1
# ✅ MSE: 2650.4043, RMSE: 51.4821, MAE: 11.5120 LR=0.01
# ✅ MSE: 2650.4043, RMSE: 51.4821, MAE: 11.5120 LR=0.01
# ✅ MSE: 2551.4658, RMSE: 50.5120, MAE: 9.2425 LR=0.05
# ✅ MSE: 2551.4846, RMSE: 50.5122, MAE: 9.2425 LR=0.5
#CHANGED LAYERS TO 128 FROM 256
# ✅ MSE: 2754.5408, RMSE: 52.4837, MAE: 15.1274 LR=0.0005
# ✅ MSE: 2751.6758, RMSE: 52.4564, MAE: 15.0302 LR=0.001
#CHANGED LAYERS TO 1024